# read

In [1]:
import pandas as pd
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/train_df.csv', encoding='cp949')
test  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


# 인코딩

In [4]:
df = pd.concat([train, test], axis=0)
df.shape

(36640, 6)

In [5]:
df['SEND_SPG_INNB'] = df['SEND_SPG_INNB'].astype('category').cat.codes.astype('str')
df['REC_SPG_INNB']  = df['REC_SPG_INNB'].astype('category').cat.codes.astype('str')

In [6]:
df = pd.get_dummies(df)

In [7]:
train = df[df['INVC_CONT'].notna()]
test  = df[df['INVC_CONT'].isna()]

train.drop('index', axis=1, inplace=True)
test.drop(['index', 'INVC_CONT'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
train.shape, test.shape

((32000, 35423), (4640, 35422))

In [9]:
train.head()

,INVC_CONT,SEND_SPG_INNB_0,SEND_SPG_INNB_1,SEND_SPG_INNB_10,SEND_SPG_INNB_100,SEND_SPG_INNB_1000,SEND_SPG_INNB_1001,SEND_SPG_INNB_1002,SEND_SPG_INNB_1003,SEND_SPG_INNB_1004,SEND_SPG_INNB_1005,SEND_SPG_INNB_1006,SEND_SPG_INNB_1007,SEND_SPG_INNB_1008,SEND_SPG_INNB_1009,SEND_SPG_INNB_101,SEND_SPG_INNB_1010,SEND_SPG_INNB_1011,SEND_SPG_INNB_1012,SEND_SPG_INNB_1013,SEND_SPG_INNB_1014,SEND_SPG_INNB_1015,SEND_SPG_INNB_1016,SEND_SPG_INNB_1017,SEND_SPG_INNB_1018,SEND_SPG_INNB_1019,SEND_SPG_INNB_102,SEND_SPG_INNB_1020,SEND_SPG_INNB_1021,SEND_SPG_INNB_1022,SEND_SPG_INNB_1023,SEND_SPG_INNB_1024,SEND_SPG_INNB_1025,SEND_SPG_INNB_1026,SEND_SPG_INNB_1027,SEND_SPG_INNB_1028,SEND_SPG_INNB_1029,SEND_SPG_INNB_103,SEND_SPG_INNB_1030,SEND_SPG_INNB_1031,...,REC_SPG_INNB_9987,REC_SPG_INNB_9988,REC_SPG_INNB_9989,REC_SPG_INNB_999,REC_SPG_INNB_9990,REC_SPG_INNB_9991,REC_SPG_INNB_9992,REC_SPG_INNB_9993,REC_SPG_INNB_9994,REC_SPG_INNB_9995,REC_SPG_INNB_9996,REC_SPG_INNB_9997,REC_SPG_INNB_9998,REC_SPG_INNB_9999,DL_GD_LCLS_NM_디지털/가전,DL_GD_LCLS_NM_생활/건강,DL_GD_LCLS_NM_식품,DL_GD_LCLS_NM_여행/문화,DL_GD_LCLS_NM_패션의류,DL_GD_LCLS_NM_화장품/미용,DL_GD_MCLS_NM_가공식품,DL_GD_MCLS_NM_건강식품,DL_GD_MCLS_NM_건강용품,DL_GD_MCLS_NM_과자,DL_GD_MCLS_NM_기타디지털/가전,DL_GD_MCLS_NM_기타식품,DL_GD_MCLS_NM_기타패션의류,DL_GD_MCLS_NM_냉동/간편조리식품,DL_GD_MCLS_NM_농산물,DL_GD_MCLS_NM_문구/사무용품,DL_GD_MCLS_NM_문화컨텐츠,DL_GD_MCLS_NM_반려동물,DL_GD_MCLS_NM_상의,DL_GD_MCLS_NM_생활용품,DL_GD_MCLS_NM_수산,DL_GD_MCLS_NM_스킨케어,DL_GD_MCLS_NM_음료,DL_GD_MCLS_NM_음반,DL_GD_MCLS_NM_주방용품,DL_GD_MCLS_NM_축산
0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# modeling

In [10]:
X = train.drop('INVC_CONT', axis=1)
y = train[['INVC_CONT']]

In [11]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 57 kB/s 


In [12]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

def objective_cat(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    params = {
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'verbose' : False
    }

    model = CatBoostRegressor(
        loss_function='RMSE', 
        random_state=0, 
        **params, 
    )

    model.fit(
        X_train, y_train,
        early_stopping_rounds=100,
        verbose=False
    )

    pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, pred, squared=False)

    return rmse

In [13]:
!pip install optuna

     |████████████████████████████████| 308 kB 5.7 MB/s 
     |████████████████████████████████| 209 kB 37.0 MB/s 
     |████████████████████████████████| 80 kB 8.9 MB/s 
     |████████████████████████████████| 75 kB 2.8 MB/s 
     |████████████████████████████████| 149 kB 46.4 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 112 kB 46.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=9537ac091a5af24288c67d61e4819f2441535fae3dde0d781b7e4f3b56c45f9e
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna

study_catboost = optuna.create_study(direction='minimize')
study_catboost.optimize(objective_cat, n_trials=50)

[I 2021-12-22 02:42:54,933] A new study created in memory with name: no-name-d1ba7bee-b2df-40d9-a37f-3308ca5fe66b
[I 2021-12-22 02:46:21,545] Trial 0 finished with value: 5.76237059010584 and parameters: {'iterations': 6018, 'od_wait': 1932, 'learning_rate': 0.4187446325717515, 'reg_lambda': 15.109386115623296, 'subsample': 0.8260731041308507, 'random_strength': 16.118039182266457, 'depth': 3, 'min_data_in_leaf': 9, 'leaf_estimation_iterations': 10}. Best is trial 0 with value: 5.76237059010584.
